In [1]:
import pandas as pd
import numpy as np
import os
import torch
os.chdir('./data')

In [8]:
train = pd.read_csv('train.csv')
val = pd.read_csv('dev.csv')
test = pd.read_csv('test.csv')

In [9]:
train.sample(5)

,id,source,sentence_1,sentence_2,label,binary-label
6414,boostcamp-sts-v1-train-6414,nsmc-sampled,진짜 잼있어요~!!,정말 재밌습니다 !!!!,4.2,1.0
3645,boostcamp-sts-v1-train-3645,petition-rtt,최저임금 차별화 적용,최저임금 차등 적용,4.0,1.0
7537,boostcamp-sts-v1-train-7537,petition-rtt,제주도 감사위원회와 제주시청 국장님땜에 청원 함부로 하면 안돼요.,제주도 감사위원회와 제주시청장이 있기에 청원을 헛되이 할 수는 없습니다.,4.0,1.0
930,boostcamp-sts-v1-train-930,slack-sampled,이번 주 1등입니다. ㅋㅋ,이번주도 금메달 목에 걸고 시작합니다.,1.0,0.0
9287,boostcamp-sts-v1-train-9287,nsmc-sampled,전혀 섹시하지도 않고 새롭지도 않고-_-,그닥 섹시해 보이지도 않으면서 액션은 초딩 수준이고..,2.4,0.0


In [38]:
import pandas as pd
import numpy as np
import os
import torch
import torch
import pytorch_lightning as pl

from args import parse_arguments
from dataloader.dataloader import Dataloader

In [46]:
args = parse_arguments()

# dataloader와 model을 정의합니다.
dataloader = Dataloader(args.model_name, args.batch_size, args.shuffle, 
                        args.train_path, args.dev_path, args.test_path, 
                        args.predict_path)
dataloader.setup()
model = torch.load(args.saved_model_path)

# gpu가 없으면 'gpus=0'을,
# gpu가 여러개면 'gpus=4'처럼 사용하실 gpu의 개수를 입력해주세요.
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
trainer = pl.Trainer(accelerator=accelerator,
                        devices=1, 
                        max_epochs=args.max_epoch, 
                        log_every_n_steps=1)

tokenizing: 100%|██████████| 550/550 [00:00<00:00, 3026.55it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [55]:
# 학습 과정에 사용된 validation 데이터에 대한 예측 값을 얻습니다.
val_dataloader = dataloader.val_dataloader()
val_predictions = trainer.predict(model=model, dataloaders=val_dataloader)


tokenizing: 100%|██████████| 1100/1100 [00:00<00:00, 3105.04it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0:   0%|          | 0/35 [00:00<?, ?it/s]

AttributeError: 'list' object has no attribute 'size'

In [ ]:

# 예측된 결과를 형식에 맞게 반올림하여 준비합니다.
val_predictions = list(round(float(i), 1) for i in torch.cat(val_predictions))

# validation 데이터를 가져와 합치고 저장합니다. 
val_data = pd.read_csv(args.dev_path)
val_data['predicted'] = val_predictions
val_data.to_csv('./data/val_output.csv', index=False)